In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import PIL.Image
import tensorflow as tf
import hashlib
import io
import os
import logging
import random
import re

from lxml import etree
#import numpy as np
#import os
import six.moves.urllib as urllib
import sys
import tarfile
#import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt


if tf.__version__ < '1.4.0':
    raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')

# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

from object_detection.utils import dataset_util
from object_detection.utils import label_map_util
from object_detection.utils.np_box_ops import iou


flags = tf.app.flags

flags.DEFINE_string('data_dir', '', 'Location of root directory')

flags.DEFINE_string('output_dir', 'C:/Users/LOLiCON/Desktop/obj_detect', 'Path output')


flags.DEFINE_string('label_map_path', 'C:/Users/LOLiCON/Desktop/obj_detect/detrac_label_map.pbtxt',
                           'Path to label map proto.')

#
FLAGS = flags.FLAGS
training_path='D:/DataSet/Insight-MVT_Annotation_Train'


data_dir_path='D:/DataSet'






C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [8]:
def dict_to_tf_example(data,
                       label_map_dict,
                       image_subdirectory,
                       bbox,
                       ignore_difficult_instances=False):
    img_path = os.path.join(image_subdirectory,data)
    
    with tf.gfile.GFile(img_path, 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = PIL.Image.open(encoded_jpg_io)
    if image.format != 'JPEG':
        raise ValueError('Image format not JPEG')
    key = hashlib.sha256(encoded_jpg).hexdigest()

    width,height = image.size
    xmins = []
    ymins = []
    xmaxs = []
    ymaxs = []
    classes = []
    classes_text = []
    truncated = []
    orientation=[]
    
    #print(img_path)
    for target in bbox.findall('target'):
        a = target.find('box')
        b = target.find('attribute')
        
        xmin= (float(a.attrib.get('left')))
        ymin= (float(a.attrib.get('top')))
        xmax= (float(a.attrib.get('width')) + xmin)
        ymax= (float(a.attrib.get('height')) +ymin)
            
        
        xmins.append(xmin / width)
        ymins.append(ymin / height)
        xmaxs.append(xmax / width)
        ymaxs.append(ymax / height)
        class_name = b.attrib.get('vehicle_type')
        classes_text.append(class_name.encode('utf8'))
        classes.append(label_map_dict[class_name])
        truncated.append(float(b.attrib.get('truncation_ratio')))
        #print(xmin,ymin,xmax,ymax)

    
    #print(xmins)
    #print(img_path)
    #print(class_name)
    feature_dict = {
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(
          data.encode('utf8')),
      'image/source_id': dataset_util.bytes_feature(
          data.encode('utf8')),
      'image/key/sha256': dataset_util.bytes_feature(key.encode('utf8')),
      'image/encoded': dataset_util.bytes_feature(encoded_jpg),
      'image/format': dataset_util.bytes_feature('jpeg'.encode('utf8')),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
        'image/object/truncated': dataset_util.float_list_feature(truncated),
  }

    example = tf.train.Example(features=tf.train.Features(feature=feature_dict))
    return example


In [9]:
def xml_tree(xml_path):
    tree = etree.parse(xml_path)
    root = tree.getroot()
    bb=''
    for weather in root.findall('sequence_attribute'):
        bb=weather.attrib.get('sence_weather')
    return bb

In [10]:
def create_tf_record(label_map_dict,
                     annotations_dir,
                     image_dir,
                     excalibur,
                     data_folder):

    

    xml_path = os.path.join(annotations_dir, data_folder+'.xml')

    print(xml_path)
    train_output_path = os.path.join(FLAGS.output_dir, 'detrac_train_only_1.record')
    val_output_path = os.path.join(FLAGS.output_dir, 'detrac_val_only_1.record')
        
        #with tf.gfile.GFile(xml_path, 'r') as fid:
    
            #xml_str = fid.read()
        #xml = etree.fromstring(xml_str)
        #data = dataset_util.recursive_parse_xml_to_dict(xml)
        #print(data['sequence'])
    txtfile=dataset_util.read_examples_list(excalibur+'/'+data_folder+'.txt')
    
    num_examples = len(txtfile)
    num_train = int(0.8 * num_examples)
    list_train_val=[]
    count_train_val=0
    for a in txtfile:
        if(count_train_val<num_train):
            list_train_val.append(1) #train 
        else:
            list_train_val.append(0) # val
        count_train_val+=1
    random.seed(42)
    random.shuffle(list_train_val)
    
    
    print(data_folder)
    aa=[]  
    img_list=[]
    tree = etree.parse(xml_path)
    root = tree.getroot()
    i=0 
    j=0
    for frame in root.findall('frame'):
        num_img= int(float(frame.attrib.get('num')))
        target_list = frame.find('target_list')
        aa.append(target_list)
        img_list.append(num_img)
        #print(num_img)
    
    try:
           
        for data in txtfile: 
            if((i+1)==img_list[j]):
                tf_example = dict_to_tf_example(data, label_map_dict, image_dir,aa[j])
                
                if(j<len(img_list)-1):
                    j=j+1
            
                if (list_train_val[i]==0):
                    writer = tf.python_io.TFRecordWriter(val_output_path)
                    writer.write(tf_example.SerializeToString())
                    writer.close()
                else:
                    writer = tf.python_io.TFRecordWriter(train_output_path)
                    writer.write(tf_example.SerializeToString())
                    writer.close()
            i=i+1
    except ValueError:
        logging.warning('Invalid example: %s, ignoring.', xml_path)

    


In [11]:

def main(_):
    data_dir = FLAGS.data_dir
    label_map_dict = label_map_util.get_label_map_dict(FLAGS.label_map_path)

   
    logging.info('Reading from DETRAC dataset.')
    
    
    annotations_dir = os.path.join(data_dir_path, 'DETRAC-Train-Annotations-XML')

  # Test images are not included in the downloaded data set, so we shall perform
  # our own split.

    
    #use 1 view only
    excalibur=data_dir_path+'/ua-text_file_front'
    
#    csv=pd.read_csv(excalibur+'/Insight-MVT_Annotation_Train.txt')
    txtfile=dataset_util.read_examples_list(excalibur+'/Insight-MVT_Annotation_Train.txt')
    
    count = 0
    
    for data_folder in txtfile:
        weather=xml_tree(os.path.join(annotations_dir, data_folder+'.xml'))
        if (weather=='sunny' or weather=='cloudy'):
            print(weather)
            count += 1
            print(count)
            print('train')
            image_dir = os.path.join(training_path, data_folder)
            create_tf_record(label_map_dict, annotations_dir,image_dir,excalibur,data_folder)
        
  # TODO(user): Write code to read in your dataset to examples variable
    
#  for example in examples:
#    tf_example = create_tf_example(example)
#    writer.write(tf_example.SerializeToString())

#  writer.close()


if __name__ == '__main__':
    tf.app.run()


sunny
1
train
D:/DataSet\DETRAC-Train-Annotations-XML\MVI_20011.xml
MVI_20011
sunny
2
train
D:/DataSet\DETRAC-Train-Annotations-XML\MVI_20012.xml
MVI_20012
sunny
3
train
D:/DataSet\DETRAC-Train-Annotations-XML\MVI_20032.xml
MVI_20032
sunny
4
train
D:/DataSet\DETRAC-Train-Annotations-XML\MVI_20033.xml
MVI_20033
sunny
5
train
D:/DataSet\DETRAC-Train-Annotations-XML\MVI_20034.xml
MVI_20034
sunny
6
train
D:/DataSet\DETRAC-Train-Annotations-XML\MVI_20035.xml
MVI_20035
sunny
7
train
D:/DataSet\DETRAC-Train-Annotations-XML\MVI_20051.xml
MVI_20051
sunny
8
train
D:/DataSet\DETRAC-Train-Annotations-XML\MVI_20052.xml
MVI_20052
sunny
9
train
D:/DataSet\DETRAC-Train-Annotations-XML\MVI_20061.xml
MVI_20061
sunny
10
train
D:/DataSet\DETRAC-Train-Annotations-XML\MVI_20062.xml
MVI_20062
sunny
11
train
D:/DataSet\DETRAC-Train-Annotations-XML\MVI_20063.xml
MVI_20063
sunny
12
train
D:/DataSet\DETRAC-Train-Annotations-XML\MVI_20064.xml
MVI_20064
sunny
13
train
D:/DataSet\DETRAC-Train-Annotations-XML\MVI_20

SystemExit: 

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
